In [1]:
import json
from tqdm.notebook import tqdm
import os

## Загрузка датасета

In [2]:
with open('novosti/dataset.json', 'r') as f:
    data = json.load(f)

len(data)

10319

In [3]:
data[0]

{'text': 'Досудебное расследование по факту покупки ЕНПФ пакета облигаций ТОО "Бузгул Аурум" было начато по инициативе Национального банка РК, сообщил директор департамента защиты прав потребителей и финансовых услуг Нацбанка Казахстана Александр Терентьев.\n"Основанием для досудебного расследования стало обращение Национального банка, письмо от 25 ноября 2016 года. Было обращение Национального банка в правоохранительные органы. Нам эта сделка показалась сомнительной, недостаточно корректной, поэтому Нацбанк 25 ноября 2016 года обратился в правоохранительные органы. Это то, что я могу озвучить на сегодня. Идёт следствие, проводится проверка", – сказал Терентьев.\n28 декабря в Нацбанке заявили, что не знают, что стало основанием для проверки ЕНПФ.\n23 декабря факт проведения проверки в АО "Единый накопительный пенсионный фонд" подтвердился. Пресс-служба Национального банка сообщила, что проверку проводят по операциям, совершённым АО "ЕНПФ" в отношении инвестирования собственных активов.

## Предобработка текстов 

In [4]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [ ]:
prep_data = []
for record in tqdm(data):
    if 'sentiment' not in record:
        continue
    
    prep_tokens = []
    for word in record['text'].split():
        cleaned_word  = ''
        for character in word.lower():
            if character in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-':
                cleaned_word += character
            elif character == '!':
                prep_tokens += ['!']
        if cleaned_word != '':
            prep_tokens += [morph.parse(cleaned_word)[0].normal_form]

    prep_record = {
        'sentiment': record['sentiment'],
        'text': ' '.join(prep_tokens)
    }
    prep_data += [prep_record]

  0%|          | 0/10319 [00:00<?, ?it/s]

In [ ]:
prep_data[0]

In [ ]:
len(prep_data)

## Анализ экспрессивов и регрессионная модель

In [ ]:
negative_set = []
neutral_set = []
positive_set = []

for record in prep_data:
    if record['sentiment'] == 'negative':
        negative_set += [record['text']]
    elif record['sentiment'] == 'neutral':
        neutral_set += [record['text']]
    else:
        positive_set += [record['text']]
len(negative_set), len(neutral_set), len(positive_set)

In [ ]:
import numpy as np

WORD = 'жаль'

logodds = []
for subset in [negative_set, neutral_set, positive_set]:
    word_freq = 0
    for text in subset:
        for word in text.split():
            if WORD == word:
                word_freq += 1
    
    subset_len = 0
    for text in subset:
        subset_len += len(text)

    logodds += [np.log(word_freq / (subset_len - word_freq))]

In [ ]:
for text in subset:
    for word in text.split():
        if word == 'жаль':
            print(text)
            break

In [ ]:
import matplotlib.pyplot as plt

sentiment = [-1, 0, 1]

k, b = np.polyfit(sentiment, logodds, 1)
y_reg_lin = []
for x in sentiment:
    y_reg_lin += [k * x + b]

a, b, c = np.polyfit(sentiment, logodds, 2)

parabolic_x = np.linspace(-1, 1)
y_reg_parabolic = []
for x in parabolic_x:
    y_reg_parabolic += [a * x ** 2 + b * x + c]

plt.scatter(sentiment, logodds)
plt.plot(sentiment, y_reg_lin, 'r--')
plt.plot(parabolic_x, y_reg_parabolic, 'g--')
plt.show()